In [2]:
#imports
import pandas as pd
import numpy as np
from itertools import combinations
import pickle
import time
import webbrowser

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder, MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, SelectFromModel, f_regression


import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#load the dataframe----bikes
url = './data/bikeshare.csv'
bikes = pd.read_csv(url, index_col='datetime', parse_dates=True)
bikes.columns=[col.lower().replace(" ","_") if col !='count' else 'total_rentals'for col in bikes.columns]
bikes.columns
bikes=bikes.drop(['atemp','registered', 'casual'],axis=1)
#feature engineering

bikes['hour']=bikes.index.hour
bikes['weekday']=bikes.index.day_name()
features=[col for col in bikes.columns if col !='total_rentals']

numerical_features=['temp','windspeed','humidity']

#feature engineering

bikes['hour']=bikes.index.hour
bikes['weekday']=bikes.index.day_name()
new_bikes=bikes.copy()
columns_to_dummify=['hour','weekday','season']
for col in columns_to_dummify:
    dummies=pd.get_dummies(bikes[col],prefix=col,drop_first=True)
    bikes=pd.concat([bikes,dummies],axis=1)
#bikes.drop(['weekday','hour'],axis=1,inplace=True)

#classifying features
cat_features=[col for col in features if col not in numerical_features]
alg_dict={'Linear Regression':LinearRegression(),
         'Random Forest': RandomForestRegressor(),
         'Ridge': Ridge(),
         'SVR':SVR()}
models=[('NN',MLPRegressor(max_iter=300,random_state=999)),
       ("boosting",GradientBoostingRegressor(random_state=999)),
        ("knn", KNeighborsRegressor()),
        ('SVR',LinearSVR())
       ]
final_models=[('NN',MLPRegressor(max_iter=600,random_state=999)),
               ('Random Forest',RandomForestRegressor())]
print('DOOOOOOOOONE')

DOOOOOOOOONE


In [ ]:
def train_test_rmse(df, feature_cols):
    X = df[feature_cols]
    y = df.total_rentals
    
    X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=123)
    
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    
    y_pred = linreg.predict(X_test)
    return np.sqrt(mean_squared_error(y_test, y_pred)),r2_score(y_test,y_pred)

def train_test_rmse_and_alg(df, feature_cols,algorithm):
    X = df[feature_cols]
    y = df.total_rentals
    
    X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=123)
    
    alg = algorithm
    alg.fit(X_train, y_train)
    
    y_pred = alg.predict(X_test)
    return np.sqrt(mean_squared_error(y_test, y_pred)),r2_score(y_test,y_pred)
def train_test_rmse_and_ss(df, feature_cols,algorithm):
    X = df[feature_cols]
    y = df.total_rentals
    
    X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=123)
    
    ss=StandardScaler()
    
    X_s_train= ss.fit_transform(X_train)
    
    X_s_test=ss.transform(X_test)
    alg=algorithm
    
    alg.fit(X_s_train, y_train)
    
    y_pred = alg.predict(X_s_test)

    return np.sqrt(mean_squared_error(y_test, y_pred)),r2_score(y_test,y_pred)

In [ ]:
st = time.time()
results_df2=pd.DataFrame(columns=['features','rmse','r2','#features','algorithm'])
count=0
for i in range(1,len(features)+1):
    for combo in list(combinations(features, i)):
        combo_list=list(combo)
        #print(combo_list,type(combo_list))
        length=len(combo_list)
#         
        count+=1
        if count%50==0:
            print('COUNT',count)
        if 'hour' in combo_list:
            #print(combo_list)
            combo_list.remove('hour')
            combo_list.extend([col for col in bikes.columns if 'hour' in col])
        if 'weekday' in combo_list:
            #print(combo_list)
            combo_list.remove('weekday')
            combo_list.extend([col for col in bikes.columns if 'weekday' in col])
        if 'season' in combo_list:
            combo_list.remove('season')
            combo_list.extend([col for col in bikes.columns if 'season' in col])
                        
        for algorithm in alg_dict.items():
            if algorithm[0] in ['Ridge','SVR'] and len(combo_list)!=1:
                #print(algorithm[0])
                rmse,r2=train_test_rmse_and_ss(bikes, combo_list,algorithm[1])
                new_row={'features':combo_list,'rmse':rmse,'r2':r2,'#features':length,\
                        'algorithm': algorithm[0]}
                results_df2 = results_df2.append(new_row, ignore_index=True)

            else:
                rmse,r2=train_test_rmse_and_alg(bikes, combo_list,algorithm[1])
                new_row={'features':combo_list,'rmse':rmse,'r2':r2,'#features':length,\
                        'algorithm': algorithm[0]}
                results_df2 = results_df2.append(new_row, ignore_index=True)

et = time.time()
  

with open('resuts_df2.pkl','wb') as file:
    pickle.dump(results_df2,file)
f'It took that many minutes {round((et-st)/60,2)} to get it done'
url = "https://www.youtube.com/watch?v=Udt-9J8nzGE"
webbrowser.open(url,new=1)

# Analyzing Results

In [ ]:
with open('resuts_df2.pkl','rb') as file:
    results= pickle.load(file)

In [ ]:
#Best algorithm
results.groupby('algorithm').rmse.min().sort_values()
##E+RandomForest is a clear winner

In [ ]:
groupy=results.groupby('#features')[['rmse','r2']].mean()
COLOR_R2 = "#69b3a2"
COLOR_RMSE = "#3399e6"

fig, ax1 = plt.subplots(figsize=(6, 6))
ax2 = ax1.twinx()

ax1.plot(groupy.index, groupy.r2, color=COLOR_R2, lw=3)
ax2.plot(groupy.index, groupy.rmse, color=COLOR_RMSE, lw=4)

ax1.set_xlabel("Number of features")
ax1.set_ylabel("R2", color=COLOR_R2, fontsize=14)
ax1.tick_params(axis="y", labelcolor=COLOR_R2)

ax2.set_ylabel("RMSE", color=COLOR_RMSE, fontsize=14)
ax2.tick_params(axis="y", labelcolor=COLOR_RMSE)

fig.suptitle("RMSE down, R2 up", fontsize=20)
fig.autofmt_xdate()
#Obvoiously, to select all features seems the best option

In [ ]:
results.sort_values('rmse').groupby('algorithm').first()
#Except for SVR all get the best results with alomst all features

In [ ]:
#Finally, just confirming our results, will anayze the score in RandomForest based on features
results[results['algorithm']=='Random Forest'].groupby('#features')[['rmse','r2']].mean()

In [ ]:
groupy_rf=results[results['algorithm']=='Random Forest'].groupby('#features')[['rmse','r2']].mean()

COLOR_R2 = "#69b3a2"
COLOR_RMSE = "#3399e6"

fig, ax1 = plt.subplots(figsize=(6, 6))
ax2 = ax1.twinx()

ax1.plot(groupy_rf.index, groupy.r2, color=COLOR_R2, lw=3)
ax2.plot(groupy_rf.index, groupy.rmse, color=COLOR_RMSE, lw=4)

ax1.set_xlabel("Number of features")
ax1.set_ylabel("R2", color=COLOR_R2, fontsize=14)
ax1.tick_params(axis="y", labelcolor=COLOR_R2)

ax2.set_ylabel("RMSE", color=COLOR_RMSE, fontsize=14)
ax2.tick_params(axis="y", labelcolor=COLOR_RMSE)

fig.suptitle("RMSE down, R2 up with Random Forest", fontsize=20)
fig.autofmt_xdate()
#Obvoiously, to select all features seems the best option

In [ ]:
#Curiosity what feature/s are left out in the best solution for LinearRegression and Ridge
for algorithm in alg_dict.keys():
    rmse_min=results[results['algorithm']==algorithm]['rmse'].min()
    list_fea=results[(results['algorithm']==algorithm)&(results['rmse']==rmse_min)]['features'].values[0]
    print(algorithm,rmse_min,[col for col in list_fea if '_' not in col and col not in features])

# Using Pipeline, crossvalidation and SelectKbest to try new models

In [ ]:
st = time.time()
results_df=pd.DataFrame(columns=['features','rmse_ytest','rmse_cv','#features','algorithm'])
new_features=[col for col in new_bikes.columns if col !='total_rentals']
numerical_features=['temp','windspeed','humidity']
categorical_features=[col for col in new_features if col not in numerical_features]

count=0

                        
for model in models:
    X=new_bikes[new_features]
    y=new_bikes.total_rentals

    X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=999)

    steps=[('columns',ColumnTransformer(
            transformers=[("cat", OneHotEncoder(categories='auto',sparse=False,handle_unknown='ignore'), categorical_features)],
            remainder="passthrough",
            )),
          ('standardization',StandardScaler()),
          ('regressor',model[1])]

    pipeline=Pipeline(steps=steps)
#             cv=KFold(n_splits=3)
#             scores=cross_val_score(pipeline,X_train,y_train,cv=cv,scoring='neg_mean_squared_error')

    pipeline.fit(X_train,y_train)
    y_preds=pipeline.predict(X_test)
    rmse=np.sqrt(mean_squared_error(y_test,y_preds))
    cv=KFold()
    scores=cross_val_score(pipeline,X_train,y_train,cv=cv,scoring='neg_mean_squared_error')


    new_row={'features':new_features,
             'rmse_ytest':rmse,
             'rmse_cv':np.sqrt(np.abs(scores)).mean(),\
             '#features':len(new_features),\
            'algorithm': model[0]}
    results_df = results_df.append(new_row, ignore_index=True)

et = time.time()


with open('resuts_df_plus.pkl','wb') as file:
    pickle.dump(results_df,file)
f'It took that many minutes {round((et-st)/60,2)} to get it done'
url = "https://www.youtube.com/watch?v=Udt-9J8nzGE"
webbrowser.open(url,new=1)

# from now on, I will only work with rf and NN.
# I will try using selectKbest just to confirm the right number of features

In [3]:
new_features=[col for col in new_bikes.columns if col !='total_rentals']
results_df=pd.DataFrame(columns=['features','rmse_cv','rmse_y_test','#features','algorithm'])
#num_features=15

X=new_bikes[new_features]
y=new_bikes.total_rentals

X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=999)

for num_features in range(44,0,-15):
    

    for model in final_models:

        steps=[('columns',ColumnTransformer(
                transformers=[("cat", OneHotEncoder(categories='auto',sparse=False,handle_unknown='ignore'), [0,3,7,8])],
                remainder="passthrough",
            )),
               ('feature_selection', SelectKBest(f_regression,k=num_features)),
              ('standardization',MinMaxScaler()),
              ('regressor',model[1])]

        pipe=Pipeline(steps=steps)
        pipe.fit(X_train,y_train)
        mask=pipe.named_steps['feature_selection'].get_support()
        column_names=pipe.named_steps['columns'].get_feature_names_out()
        selection=[''.join(a.split('__')[1]) for a,b in zip(column_names,mask) if b]
        print(num_features,len(mask),len(selection))
        y_preds=pipe.predict(X_test)
        rmse=np.sqrt(mean_squared_error(y_test,y_preds))
        cv=KFold(n_splits=3)
        scores=cross_val_score(pipe,X_train,y_train,cv=cv,scoring='neg_mean_squared_error')
        results_df.loc[len(results_df.index)] = [selection,np.sqrt(np.abs(scores).mean()),rmse,len(selection) , model[0]]
        
with open('results_nn_rf.pkl','wb') as f:
    pickle.dump(results_df,f)
print('DONE')

/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


44 44 44


/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leav

44 44 44


/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in 

...

14 44 14


/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leav

14 44 14


/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/fcbnyc/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in 

DONE


In [6]:
results_df

,features,rmse_cv,rmse_y_test,#features,algorithm
0,"[season_1, season_2, season_3, season_4, weath...",NaN,75.197869,44,NN
1,"[season_1, season_2, season_3, season_4, weath...",NaN,75.728253,44,Random Forest
2,"[season_1, season_2, season_3, weather_1, weat...",102.153560,101.586186,29,NN
3,"[season_1, season_2, season_3, weather_1, weat...",105.161217,105.560208,29,Random Forest
4,"[season_1, hour_0, hour_1, hour_2, hour_3, hou...",112.660580,110.364717,14,NN
5,"[season_1, hour_0, hour_1, hour_2, hour_3, hou...",119.283367,119.412206,14,Random Forest


# Tuning the model and analyzing the most relevant features